# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [897]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [898]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    ngrams = defaultdict(int)
    
    for s in train_text:
        w = s.split()
        if bos:
            w = ["<s>"] + w
        if eos:
            w += ["</s>"]

        # Генерация n-грамм
        for n in range(1, order + 1):
            for i in range(len(w) - n + 1):
                ngram = tuple(w[i:i + n])
                ngrams[ngram] += 1     
    
    return dict(ngrams)

In [899]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [900]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    n = len(ngram)
    prefix = ngram[:-1]
    
    count_ngram = counts.get(ngram, 0) + k
    if V is None:
    # Определяем количество уникальных слов
        V = len(set(word for ngram in counts for word in ngram))
    
    # Для 1-грамм считаем количество всех 1-грамм
    if n == 1:
        count_prefix = sum(counts[ng] for ng in counts if len(ng) == 1) + k * V
    else:
        count_prefix = counts.get(prefix, 0) + k * V
    
    return count_ngram / count_prefix

In [901]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [902]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        #генерируем все n-граммы и обновляем словарь
        self.ngrams_count = count_ngrams(train_text, order=self.order, bos=self.bos, eos=self.eos)
        if self.vocab is None:
            self.vocab = set(word for ngram in self.ngrams_count.keys() for word in ngram if len(ngram) == 1)
                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        prob = calculate_ngram_prob(ngram, self.ngrams_count, self.V, self.k)
        if prob == 0:
            return -np.inf  # Возвращаем -inf для логарифма нулевой вероятности
        else:
            return np.log(prob)
        # prob_log = np.log(prob)
        # return prob_log
    
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        ngrams_prob = 0
        for i in range(0, len(words)):
            if i + 1 < self.order:
                logprob += self.predict_ngram_log_proba(tuple(words[:i+1]))
            else:
                start = i + 1 - self.order
                end = i + 1
                logprob += self.predict_ngram_log_proba(tuple(words[start:end]))
            ngrams_prob += 1

        return logprob / ngrams_prob if ngrams_prob > 0 else -np.inf
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        log_prob_sum = 0

        for sentence in text:
            words = sentence.split()
            log_prob_sum += self.predict_log_proba(words=words)
        return np.exp(-log_prob_sum / len(text))

In [903]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (6 балла)

In [904]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=10):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 
    vocab = lm.vocab
    candidates = []
    
    for word in vocab:
        ngram = tuple(prefix[-(lm.order-1):] + [word])
        prob = lm.predict_ngram_log_proba(ngram)
        candidates.append((word, prob))
        candidates.sort(key=lambda x: x[1], reverse=True)
    return [word for word, _ in candidates[:topk]]

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?). 
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели. 

In [905]:
# Пример использования на тестовом наборе данных
prefix = ["ума"]
topk_predictions = predict_next_word(lm, prefix, topk=4)
print(f"Top-{len(topk_predictions)} продолжений для фразы {' '.join(prefix)}: {topk_predictions}")

Top-4 продолжений для фразы ума: ['прямо', 'по-моему', 'сойду', 'ума']


In [906]:
import re 
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    lines = text.split('\n')
    lines = [re.sub(r'[^\w\s]', '', line) for line in lines]
    return lines

def find_rhymes(word, rhyme_dict):
    return rhyme_dict.get(word, [])

# Обработка текста с определением рифм
def process_text_with_rhyme_labels(text):
    lines = normalize_text(text)
    rhyme_labels = defaultdict(list)
    
    for i, line in enumerate(lines):
        words = line.split()
        if words:  # Проверяем, что строка не пустая
            last_word = words[-1]
            rhyme_labels[last_word].append(last_word)
    
    return lines, rhyme_labels

# Генерация рифмованного текста
def generate_rhymed_text(lm, initial_words, rhyme_labels, rhyme_every_n=4, length=10):
    generated_text = initial_words.copy()
    
    for i in range(length):
        if i % rhyme_every_n == rhyme_every_n - 1:
            # Слово должно рифмоваться с предыдущим
            rhyme_word = generated_text[-1]
            rhyme_candidates = find_rhymes(rhyme_word, rhyme_labels)
            if rhyme_candidates:
                next_word = np.random.choice(rhyme_candidates)
            else:
                next_word = np.random.choice(list(lm.vocab))
        else:
            next_word = predict_next_word(lm, generated_text, topk=1)[0]
        
        generated_text.append(next_word)

    return ' '.join(generated_text)

def predict_next_word(lm, prefix, topk=10):
    vocab = lm.vocab
    candidates = []
    
    for word in vocab:
        ngram = tuple(prefix[-(lm.order-1):] + [word])
        prob = lm.predict_ngram_log_proba(ngram)
        candidates.append((word, prob))
    
    candidates.sort(key=lambda x: x[1], reverse=True)
    return [word for word, _ in candidates[:topk]]

In [907]:
text = """Я разобрал во тьме волшебной,
Что волею судьбы враждебной
Сей меч известен будет нам;
Что нас он обоих погубит:
Мне бороду мою отрубит,
Тебе главу; суди же сам,
Сколь важно нам приобретенье
Сего созданья злых духов!»
«Ну, что же? где тут затрудненье? —
Сказал я карле, — я готов;
Иду, хоть за пределы света».
И сосну на плечо взвалил,
А на другое для совета
Злодея брата посадил;
Пустился в дальную дорогу,
Шагал, шагал и, слава богу,
Как бы пророчеству назло,
Всё счастливо сначало шло.
За отдаленными горами
Нашли мы роковой подвал;
Я разметал его руками
И потаенный меч достал.
Но нет! судьба того хотела:
Меж нами ссора закипела —
И было, признаюсь, о чем!
Вопрос: кому владеть мечом?
Я спорил, карла горячился;
Бранились долго; наконец
Уловку выдумал хитрец,
Притих и будто бы смягчился.
«Оставим бесполезный спор, —
Сказал мне важно Черномор, —
Мы тем союз наш обесславим;
Рассудок в мире жить велит;
Судьбе решить мы предоставим,
Кому сей меч принадлежит.
К земле приникнем ухом оба
(Чего не выдумает злоба!),
И кто услышит первый звон,
Тот и владей мечом до гроба».
Сказал и лег на землю он.
Я сдуру также растянулся;
Лежу, не слышу ничего,
Смекая: обману его!
Но сам жестоко обманулся.
Злодей в глубокой тишине,
Привстав, на цыпочках ко мне
Подкрался сзади, размахнулся;
Как вихорь свистнул острый меч,
И прежде, чем я оглянулся,
Уж голова слетела с плеч —
И сверхъестественная сила
В ней жизни дух остановила.
Мой остов тернием оброс;
Вдали, в стране, людьми забвенной,
Истлел мой прах непогребенный;
Но злобный карла перенес
Меня в сей край уединенный,
Где вечно должен был стеречь
Тобой сегодня взятый меч.
О витязь! Ты храним судьбою,
Возьми его, и бог с тобою!
Быть может, на своем пути
Ты карлу-чародея встретишь —
Ах, если ты его заметишь,
Коварству, злобе отомсти!
И наконец я счастлив буду,
Спокойно мир оставлю сей —
И в благодарности моей
Твою пощечину забуду».
Песнь четвертая
Я каждый день, восстав от сна,
Благодарю сердечно бога
За то, что в наши времена
Волшебников не так уж много.
К тому же — честь и слава им! —
Женитьбы наши безопасны…
Их замыслы не так ужасны
Мужьям, девицам молодым.
Но есть волшебники другие,
Которых ненавижу я:
Улыбка, очи голубые
И голос милый — о друзья!
Не верьте им: они лукавы!
Страшитесь, подражая мне,
Их упоительной отравы,
И почивайте в тишине.
Поэзии чудесный гений,
Певец таинственных видений,
Любви, мечтаний и чертей,
Могил и рая верный житель,
И музы ветреной моей
Наперсник, пестун и хранитель!
Прости мне, северный Орфей,
Что в повести моей забавной
Теперь вослед тебе лечу
И лиру музы своенравной
Во лжи прелестной обличу.
Друзья мои, вы все слыхали,
Как бесу в древни дни злодей
Предал сперва себя с печали,
А там и души дочерей;
Как после щедрым подаяньем,
Молитвой, верой, и постом,
И непритворным покаяньем
Снискал заступника в святом;
Как умер он и как заснули
Его двенадцать дочерей:
И нас пленили, ужаснули
Картины тайных сих ночей,
Сии чудесные виденья,
Сей мрачный бес, сей божий гнев,
Живые грешника мученья
И прелесть непорочных дев.
Мы с ними плакали, бродили
Вокруг зубчатых замка стен,
И сердцем тронутым любили
Их тихий сон, их тихий плен;
Душой Вадима призывали,
И пробужденье зрели их,
И часто инокинь святых
На гроб отцовский провожали.
И что ж, возможно ль?.. нам солгали!
Но правду возвещу ли я?
Младой Ратмир, направя к югу
Нетерпеливый бег коня,
Уж думал пред закатом дня
Нагнать Русланову супругу.
Но день багряный вечерел;
Напрасно витязь пред собою
В туманы дальние смотрел:
Всё было пусто над рекою.
Зари последний луч горел
Над ярко-позлащенным бором.
Наш витязь мимо черных скал
Тихонько проезжал и взором
Ночлега меж дерев искал.
Он на долину выезжает
И видит: замок на скалах
Зубчаты стены возвышает;
Чернеют башни на углах;
И дева по стене высокой,
Как в море лебедь одинокой,
Идет, зарей освещена;
И девы песнь едва слышна
Долины в тишине глубокой.
«Ложится в поле мрак ночной;
От волн поднялся ветер хладный.
Уж поздно, путник молодой!
Укройся в терем наш отрадный.
Здесь ночью нега и покой,
А днем и шум и пированье.
Приди на дружное призванье,
Приди, о путник молодой!
У нас найдешь красавиц рой;
Их нежны речи и лобзанье.
Приди на тайное призванье,
Приди, о путник молодой!
Тебе мы с утренней зарей
Наполним кубок на прощанье.
Приди на мирное призванье,
Приди, о путник молодой!
Ложится в поле мрак ночной;
От волн поднялся ветер хладный.
Уж поздно, путник молодой!
Укройся в терем наш отрадный».
Она манит, она поет;
И юный хан уж под стеною;
Его встречают у ворот
Девицы красные толпою;
При шуме ласковых речей
Он окружен; с него не сводят
Они пленительных очей;
Две девицы коня уводят;
В чертоги входит хан младой,
За ним отшельниц милых рой;
Одна снимает шлем крылатый,
Другая кованые латы,
Та меч берет, та пыльный щит;
Одежда неги заменит
Железные доспехи брани.
Но прежде юношу ведут
К великолепной русской бане.
Уж волны дымные текут
В ее серебряные чаны,
И брызжут хладные фонтаны;
Разостлан роскошью ковер;
На нем усталый хан ложится;
Прозрачный пар над ним клубится;
Потупя неги полный взор,
Прелестные, полунагие,
В заботе нежной и немой,
Вкруг хана девы молодые
Теснятся резвою толпой.
Над рыцарем иная машет
Ветвями молодых берез,
И жар от них душистый пашет;
Другая соком вешних роз
Усталы члены прохлаждает
И в ароматах потопляет
Темнокудрявые власы.
Восторгом витязь упоенный
Уже забыл Людмилы пленной
Недавно милые красы;
Томится сладостным желаньем;
Бродящий взор его блестит,
И, полный страстным ожиданьем,
Он тает сердцем, он горит.
Но вот выходит он из бани.
Одетый в бархатные ткани,
В кругу прелестных дев, Ратмир
Садится за богатый пир.
Я не Омер: в стихах высоких
Он может воспевать один
Обеды греческих дружин
И звон и пену чаш глубоких.
Милее, по следам Парни,
Мне славить лирою небрежной
И наготу в ночной тени,
И поцелуй любови нежной!
Луною замок озарен;
Я вижу терем отдаленный,
Где витязь томный, воспаленный
Вкушает одинокий сон;
Его чело, его ланиты
Мгновенным пламенем горят;
Его уста полуоткрыты
Лобзанье тайное манят;
Он страстно, медленно вздыхает,
Он видит их — и в пылком сне
Покровы к сердцу прижимает.
Но вот в глубокой тишине
Дверь отворилась: пол ревнивый
Скрыпит под ножкой торопливой,
И при серебряной луне
Мелькнула дева. Сны крылаты,
Сокройтесь, отлетите прочь!
Проснись — твоя настала ночь!
Проснися — дорог миг утраты!..
Она подходит, он лежит
И в сладострастной неге дремлет;
Покров его с одра скользит,
И жаркий пух чело объемлет.
В молчаньи дева перед ним
Стоит недвижно, бездыханна,
Как лицемерная Диана
Пред милым пастырем своим;
И вот она, на ложе хана
Коленом опершись одним,
Вздохнув, лицо к нему склоняет
С томленьем, с трепетом живым,
И сон счастливца прерывает
Лобзаньем страстным и немым…
Но, други, девственная лира
Умолкла под моей рукой;
Слабеет робкий голос мой —
Оставим юного Ратмира;
Не смею песней продолжать:
Руслан нас должен занимать,
Руслан, сей витязь беспримерный,
В душе герой, любовник верный.
Упорным боем утомлен,
Под богатырской головою
Он сладостный вкушает сон.
Но вот уж раннею зарею
Сияет тихий небосклон;
Всё ясно; утра луч игривый
Главы косматый лоб златит.
Руслан встает, и конь ретивый
Уж витязя стрелою мчит.
И дни бегут; желтеют нивы;
С дерев спадает дряхлый лист;
В лесах осенний ветра свист
Певиц пернатых заглушает;
Тяжелый, пасмурный туман
Нагие холмы обвивает;
Зима приближилась — Руслан
Свой путь отважно продолжает
На дальный север; с каждым днем
Преграды новые встречает:
То бьется он с богатырем,
То с ведьмою, то с великаном,
То лунной ночью видит он,
Как будто сквозь волшебный сон,
Окружены седым туманом,
Русалки, тихо на ветвях
Качаясь, витязя младого
С улыбкой хитрой на устах
Манят, не говоря ни слова…
Но тайным промыслом храним,
Бесстрашный витязь невредим;
В его душе желанье дремлет,
Он их не видит, им не внемлет,
Одна Людмила всюду с ним.
Но между тем, никем не зрима,
От нападений колдуна
Волшебной шапкою хранима,
Что делает моя княжна,
Моя прекрасная Людмила?
Она, безмолвна и уныла,
Одна гуляет по садам,
О друге мыслит и вздыхает,
Иль, волю дав своим мечтам,
К родимым киевским полям
В забвенье сердца улетает;
Отца и братьев обнимает,
Подружек видит молодых
И старых мамушек своих —
Забыты плен и разлученье!
Но вскоре бедная княжна
Свое теряет заблужденье
И вновь уныла и одна.
Рабы влюбленного злодея,
И день и ночь, сидеть не смея,
Меж тем по замку, по садам
Прелестной пленницы искали,
Метались, громко призывали,
Однако всё по пустякам.
Людмила ими забавлялась:
В волшебных рощах иногда
Без шапки вдруг она являлась
И кликала: «Сюда, сюда!»
И все бросались к ней толпою;
Но в сторону — незрима вдруг —
Она неслышною стопою
От хищных убегала рук.
Везде всечасно замечали
Ее минутные следы:
То позлащенные плоды
На шумных ветвях исчезали,
То капли ключевой воды
На луг измятый упадали:
Тогда наверно в замке знали,
Что пьет иль кушает княжна.
На ветвях кедра иль березы
Скрываясь по ночам, она
Минутного искала сна —
Но только проливала слезы,
Звала супруга и покой,
Томилась грустью и зевотой,
И редко, редко пред зарей,
Склонясь ко древу головой,
Дремала тонкою дремотой;
Едва редела ночи мгла,
Людмила к водопаду шла
Умыться хладною струею:
Сам карла утренней порою
Однажды видел из палат,
Как под невидимой рукою
Плескал и брызгал водопад.
С своей обычною тоскою
До новой ночи, здесь и там,
Она бродила по садам:
Нередко под вечер слыхали
Ее приятный голосок;
Нередко в рощах поднимали
Иль ею брошенный венок,
Или клочки персидской шали,
Или заплаканный платок.
Жестокой страстью уязвленный,
Досадой, злобой омраченный,
Колдун решился наконец
Поймать Людмилу непременно.
Так Лемноса хромой кузнец,
Прияв супружеский венец
Из рук прелестной Цитереи,
Раскинул сеть ее красам,
Открыв насмешливым богам
Киприды нежные затеи…
Скучая, бедная княжна
В прохладе мраморной беседки
Сидела тихо близ окна
И сквозь колеблемые ветки
Смотрела на цветущий луг.
Вдруг слышит — кличут: «Милый друг!»
И видит верного Руслана.
Его черты, походка, стан;
Но бледен он, в очах туман,
И на бедре живая рана —
В ней сердце дрогнуло. «Руслан!
Руслан!.. он точно!» И стрелою
К супругу пленница летит,
В слезах, трепеща, говорит:
«Ты здесь… ты ранен… что с тобою?»
Уже достигла, обняла:
О ужас… призрак исчезает!
Княжна в сетях; с ее чела
На землю шапка упадает.
Хладея, слышит грозный крик:
«Она моя!» — и в тот же миг
Зрит колдуна перед очами.
Раздался девы жалкий стон,
Падет без чувств — и дивный сон
Объял несчастную крылами.
Что будет с бедною княжной!
О страшный вид: волшебник хилый
Ласкает дерзостной рукой
Младые прелести Людмилы!
Ужели счастлив будет он?
Чу… вдруг раздался рога звон,
И кто-то карлу вызывает.
В смятеньи, бледный чародей
На деву шапку надевает;
Трубят опять; звучней, звучней!
И он летит к безвестной встрече,
Закинув бороду за плечи.
Песнь пятая
Ах, как мила моя княжна!
Мне нрав ее всего дороже:
Она чувствительна, скромна,
Любви супружеской верна,
Немножко ветрена… так что же?
Еще милее тем она.
Всечасно прелестию новой
Умеет нас она пленить;
Скажите: можно ли сравнить
Ее с Дельфирою суровой?
Одной — судьба послала дар
Обворожать сердца и взоры;
Ее улыбка, разговоры
Во мне любви рождают жар.
А та — под юбкою гусар,
Лишь дайте ей усы да шпоры!
Блажен, кого под вечерок
В уединенный уголок
Моя Людмила поджидает
И другом сердца назовет;
Но, верьте мне, блажен и тот,
Кто от Дельфиры убегает
И даже с нею незнаком.
Да, впрочем, дело не о том!
Но кто трубил? Кто чародея
На сечу грозну вызывал?
Кто колдуна перепугал?
Руслан. Он, местью пламенея,
Достиг обители злодея.
Уж витязь под горой стоит,
Призывный рог, как буря, воет,
Нетерпеливый конь кипит
И снег копытом мочным роет.
Князь карлу ждет. Внезапно он
По шлему крепкому стальному
Рукой незримой поражен;
Удар упал подобно грому;
Руслан подъемлет смутный взор
И видит — прямо над главою —
С подъятой, страшной булавою
Летает карла Черномор.
Щитом покрывшись, он нагнулся,
Мечом потряс и замахнулся;
Но тот взвился под облака;
На миг исчез — и свысока
Шумя летит на князя снова.
Проворный витязь отлетел,
И в снег с размаха рокового
Колдун упал — да там и сел;
Руслан, не говоря ни слова,
С коня долой, к нему спешит,
Поймал, за бороду хватает,
Волшебник силится, кряхтит
И вдруг с Русланом улетает…
Ретивый конь вослед глядит;
Уже колдун под облаками;
На бороде герой висит;
Летят над мрачными лесами,
Летят над дикими горами,
Летят над бездною морской;
От напряженья костенея,
Руслан за бороду злодея
Упорной держится рукой.
Меж тем, на воздухе слабея
И силе русской изумясь,
Волшебник гордому Руслану
Коварно молвит: «Слушай, князь!
Тебе вредить я перестану;
Младое мужество любя,
Забуду всё, прощу тебя,
Спущусь — но только с уговором…»
«Молчи, коварный чародей! —
Прервал наш витязь: — с Черномором,
С мучителем жены своей,
Руслан не знает договора!
Сей грозный меч накажет вора.
Лети хоть до ночной звезды,
А быть тебе без бороды!»
Боязнь объемлет Черномора;
В досаде, в горести немой,
Напрасно длинной бородой
Усталый карла потрясает:
Руслан ее не выпускает
И щиплет волосы порой.
Два дни колдун героя носит,
На третий он пощады просит:
«О рыцарь, сжалься надо мной;
Едва дышу; нет мочи боле;
Оставь мне жизнь, в твоей я воле;
Скажи — спущусь, куда велишь… »
«Теперь ты наш: ага, дрожишь!
Смирись, покорствуй русской силе!
Неси меня к моей Людмиле».
Смиренно внемлет Черномор;
Домой он с витязем пустился;
Летит — и мигом очутился
Среди своих ужасных гор.
Тогда Руслан одной рукою
Взял меч сраженной головы
И, бороду схватив другою,
Отсек ее, как горсть травы.
«Знай наших! — молвил он жестоко, —
Что, хищник, где твоя краса?
Где сила?» — и на шлем высокий
Седые вяжет волоса;
Свистя зовет коня лихого;
Веселый конь летит и ржет;
Наш витязь карлу чуть живого
В котомку за седло кладет,
А сам, боясь мгновенья траты,
Спешит на верх горы крутой,
Достиг, и с радостной душой
Летит в волшебные палаты.
Вдали завидя шлем брадатый,
Залог победы роковой,
Пред ним арапов чудный рой,
Толпы невольниц боязливых,
Как призраки, со всех сторон
Бегут — и скрылись. Ходит он
Один средь храмин горделивых,
Супругу милую зовет —
Лишь эхо сводов молчаливых
Руслану голос подает;
В волненье чувств нетерпеливых
Он отворяет двери в сад —
Идет, идет — и не находит;
Кругом смущенный взор обводит —
Всё мертво: рощицы молчат,
Беседки пусты; на стремнинах,
Вдоль берегов ручья, в долинах,
Нигде Людмилы следу нет,
И ухо ничего не внемлет.
Внезапный князя хлад объемлет,
В очах его темнеет свет,
В уме возникли мрачны думы…
«Быть может, горесть… плен угрюмый…
Минута… волны…» В сих мечтах
Он погружен. С немой тоскою
Поникнул витязь головою;
Его томит невольный страх;
Недвижим он, как мертвый камень;
Мрачится разум; дикий пламень
И яд отчаянной любви
Уже текут в его крови.
Казалось — тень княжны прекрасной
Коснулась трепетным устам…
И вдруг, неистовый, ужасный,
Стремится витязь по садам;
Людмилу с воплем призывает,
С холмов утесы отрывает,
Всё рушит, всё крушит мечом —
Беседки, рощи упадают,
Древа, мосты в волнах ныряют,
Степь обнажается кругом!
Далеко гулы повторяют
И рев, и треск, и шум, и гром;
Повсюду меч звенит и свищет,
Прелестный край опустошен —
Безумный витязь жертвы ищет,
С размаха вправо, влево он
Пустынный воздух рассекает…
И вдруг — нечаянный удар
С княжны невидимой сбивает
Прощальный Черномора дар…
Волшебства вмиг исчезла сила:
В сетях открылася Людмила!
Не веря сам своим очам,
Нежданным счастьем упоенный,
Наш витязь падает к ногам
Подруги верной, незабвенной,
Целует руки, сети рвет,
Любви, восторга слезы льет,
Зовет ее — но дева дремлет,
Сомкнуты очи и уста,
И сладострастная мечта
Младую грудь ее подъемлет.
Руслан с нее не сводит глаз,
Его терзает вновь кручина…
Но вдруг знакомый слышит глас,
Глас добродетельного Финна:
«Мужайся, князь! В обратный путь
Ступай со спящею Людмилой;
Наполни сердце новой силой,
Любви и чести верен будь.
Небесный гром на злобу грянет,
И воцарится тишина —
И в светлом Киеве княжна
Перед Владимиром восстанет
От очарованного сна».
Руслан, сим гласом оживленный,
Берет в объятия жену,
И тихо с ношей драгоценной
Он оставляет вышину
И сходит в дол уединенный.
В молчаньи, с карлой за седлом,
Поехал он своим путем;
В его руках лежит Людмила
Свежа, как вешняя заря,
И на плечо богатыря
Лицо спокойное склонила.
Власами, свитыми в кольцо,
Пустынный ветерок играет;
Как часто грудь ее вздыхает!
Как часто тихое лицо
Мгновенной розою пылает!
Любовь и тайная мечта
Русланов образ ей приносят,
И с томным шопотом уста
Супруга имя произносят…
В забвеньи сладком ловит он
Её волшебное дыханье,
Улыбку, слезы, нежный стон
И сонных персей волнованье…
Меж тем, по долам, по горам,
И в белый день, и по ночам,
Наш витязь едет непрестанно.
Еще далек предел желанный,
А дева спит. Но юный князь,
Бесплодным пламенем томясь,
Ужель, страдалец постоянный,
Супругу только сторожил
И в целомудренном мечтанье,
Смирив нескромное желанье,
Свое блаженство находил?
Монах, который сохранил
Потомству верное преданье
О славном витязе моем,
Нас уверяет смело в том:
И верю я! Без разделенья
Унылы, грубы наслажденья:
Мы прямо счастливы вдвоем.
Пастушки, сон княжны прелестной
Не походил на ваши сны,
Порой томительной весны,
На мураве, в тени древесной.
Я помню маленький лужок
Среди березовой дубравы,
Я помню темный вечерок,
Я помню Лиды сон лукавый…
Ах, первый поцелуй любви,
Дрожащий, легкий, торопливый,
Не разогнал, друзья мои,
Ее дремоты терпеливой…
Но полно, я болтаю вздор!
К чему любви воспоминанье?
Ее утеха и страданье
Забыты мною с давних пор;
Теперь влекут мое вниманье
Княжна, Руслан и Черномор.
Пред ними стелется равнина,
Где ели изредка взошли;
И грозного холма вдали
Чернеет круглая вершина
Небес на яркой синеве.
Руслан глядит — и догадался,
Что подъезжает к голове;
Быстрее борзый конь помчался;
Уж видно чудо из чудес;
Она глядит недвижным оком;
Власы ее как черный лес,
Поросший на челе высоком;
Ланиты жизни лишены,
Свинцовой бледностью покрыты;
Уста огромные открыты,
Огромны зубы стеснены…
Над полумертвой головою
Последний день уж тяготел.
К ней храбрый витязь прилетел
С Людмилой, с карлой за спиною.
Он крикнул: «Здравствуй, голова!
Я здесь! наказан твой изменник!
Гляди: вот он, злодей наш пленник!»
И князя гордые слова
Ее внезапно оживили,
На миг в ней чувство разбудили,
Очнулась будто ото сна,
Взглянула, страшно застонала…
Узнала витязя она
И брата с ужасом узнала.
Надулись ноздри; на щеках
Багровый огнь еще родился,
И в умирающих глазах
Последний гнев изобразился.
В смятенье, в бешенстве немом
Она зубами скрежетала
И брату хладным языком
Укор невнятный лепетала…
Уже ее в тот самый час
Кончалось долгое страданье:
Чела мгновенный пламень гас,
Слабело тяжкое дыханье,
Огромный закатился взор,
И вскоре князь и Черномор
Узрели смерти содроганье…
Она почила вечным сном.
В молчанье витязь удалился;
Дрожащий карлик за седлом
Не смел дышать, не шевелился
И чернокнижным языком
Усердно демонам молился.
На склоне темных берегов
Какой-то речки безымянной,
В прохладном сумраке лесов,
Стоял поникшей хаты кров,
Густыми соснами венчанный.
В теченьи медленном река
Вблизи плетень из тростника
Волною сонной омывала
И вкруг него едва журчала
При легком шуме ветерка.
Долина в сих местах таилась,
Уединенна и темна;
И там, казалось, тишина
С начала мира воцарилась.
Руслан остановил коня.
Всё было тихо, безмятежно;
От рассветающего дня
Долина с рощею прибрежной
Сквозь утренний сияла дым.
Руслан на луг жену слагает,
Садится близ нее, вздыхает
С уныньем сладким и немым;
И вдруг он видит пред собою
Смиренный парус челнока
И слышит песню рыбака
Над тихоструйною рекою.
Раскинув невод по волнам,
Рыбак, на весла наклоненный,
Плывет к лесистым берегам,
К порогу хижины смиренной.
И видит добрый князь Руслан:
Челнок ко брегу приплывает;
Из темной хаты выбегает
Младая дева; стройный стан,
Власы, небрежно распущенны,
Улыбка, тихий взор очей,
И грудь, и плечи обнаженны,
Всё мило, всё пленяет в ней.
И вот они, обняв друг друга,
Садятся у прохладных вод,
И час беспечного досуга
Для них с любовью настает.
Но в изумленьи молчаливом
Кого же в рыбаке счастливом
Наш юный витязь узнает?
Хазарский хан, избранный славой,
Ратмир, в любви, в войне кровавой
Его соперник молодой,
Ратмир в пустыне безмятежной
Людмилу, славу позабыл
И им навеки изменил
В объятиях подруги нежной.
Герой приближился, и вмиг
Отшельник узнает Руслана,
Встает, летит. Раздался крик…
И обнял князь младого хана.
«Что вижу я? — спросил герой, —
Зачем ты здесь, зачем оставил
Тревоги жизни боевой
И меч, который ты прославил?»
«Мой друг, — ответствовал рыбак, —
Душе наскучил бранной славы
Пустой и гибельный призрак.
Поверь: невинные забавы,
Любовь и мирные дубравы
Милее сердцу во сто крат.
Теперь, утратив жажду брани,
Престал платить безумству дани,
И, верным счастием богат,
Я всё забыл, товарищ милый,
Всё, даже прелести Людмилы».
«Любезный хан, я очень рад! —
Сказал Руслан, — она со мною».
«Возможно ли, какой судьбою?
Что слышу? Русская княжна…
Она с тобою, где ж она?
Позволь… но нет, боюсь измены;
Моя подруга мне мила;
Моей счастливой перемены
Она виновницей была;
Она мне жизнь, она мне радость!
Она мне возвратила вновь
Мою утраченную младость,
И мир, и чистую любовь.
Напрасно счастье мне сулили
Уста волшебниц молодых;
Двенадцать дев меня любили:
Я для нее покинул их;
Оставил терем их веселый,
В тени хранительных дубров;
Сложил и меч и шлем тяжелый,
Забыл и славу и врагов.
Отшельник, мирный и безвестный,
Остался в счастливой глуши,
С тобой, друг милый, друг прелестный,
С тобою, свет моей души!»
Пастушка милая внимала
Друзей открытый разговор
И, устремив на хана взор,
И улыбалась и вздыхала.
Рыбак и витязь на брегах
До темной ночи просидели
С душой и сердцем на устах —
Часы невидимо летели.
Чернеет лес, темна гора;
Встает луна — всё тихо стало.
Герою в путь давно пора —
Накинув тихо покрывало
На деву спящую, Руслан
Идет и на коня садится;
Задумчиво безмолвный хан
Душой вослед ему стремится,
Руслану счастия, побед
И славы и любви желает…
И думы гордых, юных лет
Невольной грустью оживляет…
Зачем судьбой не суждено
Моей непостоянной лире
Геройство воспевать одно
И с ним (незнаемые в мире)
Любовь и дружбу старых лет?
Печальной истины поэт,
Зачем я должен для потомств
Порок и злобу обнажать
И тайны козни вероломства
В правдивых песнях обличать?
Княжны искатель недостойный,
Охоту к славе потеряв,
Никем не знаемый, Фарлаф
В пустыне дальней и спокойной
Скрывался и Наины ждал.
И час торжественный настал.
К нему волшебница явилась,
Вещая: «Знаешь ли меня?
Ступай за мной; седлай коня!»
И ведьма кошкой обратилась;
Оседлан конь, она пустилась;
Тропами мрачными дубрав
За нею следует Фарлаф.
Долина тихая дремала,
В ночной одетая туман,
Луна во мгле перебегала
Из тучи в тучу и курган
Мгновенным блеском озаряла.
Под ним в безмолвии Руслан
Сидел с обычною тоскою
Пред усыпленною княжною.
Глубоку думу думал он,
Мечты летели за мечтами,
И неприметно веял сон
Над ним холодными крылами.
На деву смутными очами
В дремоте томной он взглянул
И, утомленною главою
Склонясь к ногам ее, заснул.
И снится вещий сон герою:
Он видит, будто бы княжна
Над страшной бездны глубиною
Стоит недвижна и бледна…
И вдруг Людмила исчезает,
Стоит один над бездной он…
Знакомый глас, призывный стон
Из тихой бездны вылетает…
Руслан стремится за женой;
Стремглав летит во тьме глубокой.
И видит вдруг перед собой:
Владимир, в гриднице высокой,
В кругу седых богатырей,
Между двенадцатью сынами,
С толпою названных гостей
Сидит за браными столами.
И так же гневен старый князь,
Как в день ужасный расставанья,
И все сидят не шевелясь,
Не смея перервать молчанья.
Утих веселый шум гостей,
Не ходит чаша круговая…
И видит он среди гостей
В бою сраженного Рогдая:
Убитый, как живой, сидит;
Из опененного стакана
Он, весел, пьет и не глядит
На изумленного Руслана.
Князь видит и младого хана,
Друзей и недругов… и вдруг
Раздался гуслей беглый звук
И голос вещего Баяна,
Певца героев и забав.
Вступает в гридницу Фарлаф,
Ведет он за руку Людмилу;
Но старец, с места не привстав,
Молчит, склонив главу унылу,
Князья, бояре — все молчат,
Душевные движенья кроя.
И всё исчезло — смертный хлад
Объемлет спящего героя.
"""

In [908]:
normalized_lines, rhyme_labels = process_text_with_rhyme_labels(text)

In [909]:
lm = NgramLM(order=2, k=2)
lm.fit(normalized_lines)

In [910]:
initial_sentence = ["я", "разобрал", "твой"]
generated_text = generate_rhymed_text(lm, initial_sentence, rhyme_labels, rhyme_every_n=5, length=10)
print("Generated rhymed text:", generated_text)

Generated rhymed text: я разобрал твой изменник гляди вот уж вздохнув лицо спокойное склонила власами утесы
